In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 30.1 MB/s 
     |████████████████████████████████| 596 kB 70.5 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 6.6 MB 72.2 MB/s 
     |████████████████████████████████| 895 kB 86.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 30.6 MB/s 
     |████████████████████████████████| 1.1 MB 25.5 MB/s 
     |████████████████████████████████| 212 kB 72.5 MB/s 
     |████████████████████████████████| 136 kB 62.6 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
     |████████████████████████████████| 144 kB 70.1 MB/s 
     |████████████████████████████████| 271 kB 74.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import ast

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/NLP_Proj/data_with_keywords_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/NLP_Proj/data_with_keywords_test.csv')

In [ ]:
train_data_truncated = train_data[train_data['verdict'].isin(['asshole', 'not the asshole'])]
test_data_truncated = test_data[test_data['verdict'].isin(['asshole', 'not the asshole'])]
len(test_data_truncated)

1002

In [ ]:
test_data_truncated.head()

,Unnamed: 0,Unnamed: 0.1,title,body,num_comments,verdict,word_count,has_tldr,has_background,has_update,has_edit,keywords,title_key_word
0,32,32,aita for being mad @ my gf for agreeing to tak...,(background: our school has a rule that undrec...,13.0,not the asshole,264,False,True,False,False,"['take another girl', 'prom', 'mad', 'gf', 'ai...","['gf agreeing', 'aita', 'prom', 'mad', 'girl']"
1,5205,5205,aita for demanding a product for the price adv...,tl;dr - bought something on amazon for \~$10 d...,33.0,asshole,346,False,False,False,False,"['price advertised', 'amazon )?', 'product', '...","['aita demanding', 'product price', 'amazon', ..."
2,7963,7963,aita for telling my mother off because she kee...,"i'm 16, still living at home with my mother. \...",8.0,not the asshole,273,False,False,False,False,"['keeps commenting', 'telling', 'mother', 'eat...","['aita telling', 'keeps commenting', 'eat', 'm..."
3,32594,32594,aita for refusing to take an elderly man’s blo...,i recently started a new job as a caregiver. i...,63.0,not the asshole,330,False,False,False,False,"['elderly man ’', 'blood sugar', '’', 'take', ...","['refusing elderly', 'man blood', 'aita', 'sug..."
4,30160,30160,aita for not tipping a bathroom attendant?,me (23f) and a few of my friends went to a bar...,91.0,not the asshole,301,False,False,False,True,"['bathroom attendant', 'tipping', 'aita']","['aita tipping', 'bathroom', 'attendant', 'tip..."


In [ ]:
test_data_truncated = test_data_truncated[['title', 'body', 'verdict', 'keywords']]

In [ ]:
test_data_truncated = test_data_truncated.fillna('')
test_data_truncated

,title,body,verdict,keywords
0,aita for being mad @ my gf for agreeing to tak...,(background: our school has a rule that undrec...,not the asshole,"['take another girl', 'prom', 'mad', 'gf', 'ai..."
1,aita for demanding a product for the price adv...,tl;dr - bought something on amazon for \~$10 d...,asshole,"['price advertised', 'amazon )?', 'product', '..."
2,aita for telling my mother off because she kee...,"i'm 16, still living at home with my mother. \...",not the asshole,"['keeps commenting', 'telling', 'mother', 'eat..."
3,aita for refusing to take an elderly man’s blo...,i recently started a new job as a caregiver. i...,not the asshole,"['elderly man ’', 'blood sugar', '’', 'take', ..."
4,aita for not tipping a bathroom attendant?,me (23f) and a few of my friends went to a bar...,not the asshole,"['bathroom attendant', 'tipping', 'aita']"
...,...,...,...,...
997,aita for telling my mom she was rude and overs...,throwaway\n\ni (27 f) was recently left in a d...,asshole,"['telling', 'rude', 'overstepping', 'mom', 'ai..."
998,aita for cutting contact with a close friend?,i have agroup of friends i met in highschool. ...,not the asshole,"['cutting contact', 'close friend', 'aita']"
999,aita for eating my snacks after a volleyball g...,i recently discovered this subreddit and have ...,not the asshole,"['teammates brought snacks', 'volleyball game'..."
1000,aita for refusing to take an order someone els...,this happened a few months ago. i work as a pi...,asshole,"['order someone else forgot half', 'take', 're..."


In [ ]:
print(data_truncated.iloc[37511].title)
print(data_truncated.iloc[37511].keywords)

aita for being disrespectful with my mum over her hating anime?
['hating anime', 'mum', 'disrespectful', 'aita']


In [ ]:
TOKENS_THRESHOLD = 200
def filter_fn(row):
    if len(row['body'].split()) > TOKENS_THRESHOLD:
        return True
    else:
        return False

data_truncated = test_data_truncated[test_data_truncated.apply(filter_fn, axis=1)]
data_truncated = data_truncated.reset_index()
data_truncated

,index,title,body,verdict,keywords
0,0,aita for asking my boyfriend to drive me to th...,"i am very, very sick right now with the flu. i...",not the asshole,"['hospital', 'drive', 'boyfriend', 'asking', '..."
1,1,aita for buying a phone off of my close friend...,my friend let’s call her zoe & her ex let’s ca...,not the asshole,"['close friend ’', 'ex boyfriend', 'phone', 'b..."
2,2,aita for not answering my phone to my mom duri...,3 months ago i was going to a sleepover with m...,asshole,"['sleepover', 'phone', 'mom', 'answering', 'ai..."
3,3,aita for asking my girlfriend to straighten he...,so some backstory. my girlfriend has really be...,asshole,"['straighten', 'hair', 'girlfriend', 'asking',..."
4,4,aita for telling my girlfriend to stop eating ...,"weird title, and this’ll sound fake, but it’s ...",asshole,"['stop eating “ baby snacks ”?', 'telling', 'g..."
...,...,...,...,...,...
29328,37507,aita for letting someone sit in my seat at the...,i went to watch a play with my girlfriend last...,asshole,"['letting someone sit', 'theatre', 'seat', 'ai..."
29329,37508,aita for venting in a kind of nasty way about ...,so i was hanging out with my friend and i star...,asshole,"['nasty way', 'venting', 'kind', 'classmates',..."
29330,37509,wibta if i asked my neighbor to help with the ...,tl;dr at the bottom.\n\n\n**i will also likely...,asshole,"['wibta', 'tree', 'removing', 'neighbor', 'hom..."
29331,37510,aita for not wanting my fiancée to invite her ...,"my fiancée and i have been dating 3 years, eng...",asshole,"['wedding', 'wanting', 'stepdaughter', 'invite..."


In [ ]:
data_small = data_truncated.sample(n=25000)
data_small

,index,title,body,verdict,keywords
7034,9062,aita for taking alcohol even if we all paid fo...,it was my friend's 25th birthday last week and...,not the asshole,"['taking alcohol even', 'paid', 'aita']"
1802,2342,aita for trying to date one of my closest frie...,some backstory:\n\nabout 3 and a half years on...,asshole,"['closest friend ’', 'past abuser', 'date one'..."
24692,31559,wibta if i accepted my grandmothers inheritanc...,"i am not from an english speaking country, so ...",not the asshole,"['grandmothers inheritance leaving', 'wibta', ..."
318,401,aita for not changing my wedding date?,we got engaged in late 2018. my stbh and i cou...,not the asshole,"['wedding date', 'changing', 'aita']"
23800,30435,"aita for telling my ex ""why would you care""?","for context, we are still going to the same sc...",asshole,"['care ""?', 'would', 'telling', 'ex', 'aita']"
...,...,...,...,...,...
400,509,wibta if i stopped being friends with someone ...,wibta if i (34f) stopped being friends with my...,not the asshole,"['29 years', 'wibta', 'wedding', 'stopped', 's..."
28183,36039,aita for offering waaay low on some guitar gea...,so my coworker found out that my son and i are...,asshole,"['offering waaay low', 'guitar gear', 'coworke..."
20644,26413,"aita (d&d edition) for ""sharing"" a bad guy wit...",disclaimer: if you don't play dungeons and dra...,not the asshole,"['entire party', 'bad guy', 'sharing', 'editio..."
14384,18410,wibta if i reported my parents dangerous dog,"not sure if this is against the rules or not, ...",not the asshole,"['parents dangerous dog', 'wibta', 'reported']"


In [ ]:
data_small = data_small.reset_index(drop = True)
data_small = data_small[['title', 'body', 'verdict', 'keywords']]
data_small

,title,body,verdict,keywords
0,aita for taking alcohol even if we all paid fo...,it was my friend's 25th birthday last week and...,not the asshole,"['taking alcohol even', 'paid', 'aita']"
1,aita for trying to date one of my closest frie...,some backstory:\n\nabout 3 and a half years on...,asshole,"['closest friend ’', 'past abuser', 'date one'..."
2,wibta if i accepted my grandmothers inheritanc...,"i am not from an english speaking country, so ...",not the asshole,"['grandmothers inheritance leaving', 'wibta', ..."
3,aita for not changing my wedding date?,we got engaged in late 2018. my stbh and i cou...,not the asshole,"['wedding date', 'changing', 'aita']"
4,"aita for telling my ex ""why would you care""?","for context, we are still going to the same sc...",asshole,"['care ""?', 'would', 'telling', 'ex', 'aita']"
...,...,...,...,...
24995,wibta if i stopped being friends with someone ...,wibta if i (34f) stopped being friends with my...,not the asshole,"['29 years', 'wibta', 'wedding', 'stopped', 's..."
24996,aita for offering waaay low on some guitar gea...,so my coworker found out that my son and i are...,asshole,"['offering waaay low', 'guitar gear', 'coworke..."
24997,"aita (d&d edition) for ""sharing"" a bad guy wit...",disclaimer: if you don't play dungeons and dra...,not the asshole,"['entire party', 'bad guy', 'sharing', 'editio..."
24998,wibta if i reported my parents dangerous dog,"not sure if this is against the rules or not, ...",not the asshole,"['parents dangerous dog', 'wibta', 'reported']"


In [ ]:
data_small['verdict'].value_counts()

not the asshole    12913
asshole            12087
Name: verdict, dtype: int64

In [ ]:
data_truncated['verdict'].value_counts()

not the asshole    15200
asshole            14133
Name: verdict, dtype: int64

In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', 
                                          bos_token='<|startofprompt|>', 
                                          eos_token='<|endoftext|>', 
                                          pad_token='<|pad|>',
                                         )


tokenizer.encode("Sample Text")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[36674, 8255]

In [ ]:
len(tokenizer)

50259

In [ ]:
tokenizer.add_tokens(['<|endofprompt|>', '<|startofverdict|>', '<|endofverdict|>', '<|sok|>', '<|eok|>', '<|startoftext|>'], special_tokens=True)
tokenizer.convert_tokens_to_ids(['<|endofprompt|>', '<|startofverdict|>', '<|endofverdict|>', '<|sok|>', '<|eok|>', '<|startoftext|>'])

[50259, 50260, 50261, 50262, 50263, 50264]

In [ ]:
len(tokenizer)

50265

In [ ]:
def join_keywords(keywords, randomize=True):
    N = len(keywords)

    #random sampling and shuffle
    if randomize: 
        # M = random.choice(range(N+1))
        keywords = keywords[:N]
        random.shuffle(keywords)

    return ','.join(keywords)

In [ ]:
train_titles = train_data_truncated.title.copy()
train_verdicts = train_data_truncated.verdict.copy()
train_stories = train_data_truncated.body.copy()
train_keywords = train_data_truncated.keywords.copy()
train_keywords = pd.Series(list(map(lambda x: ast.literal_eval(x), train_keywords)))
train_keywords = list(map(lambda x: join_keywords(x), train_keywords))


test_titles = test_data_truncated.title.copy()
test_verdicts = test_data_truncated.verdict.copy()
test_stories = test_data_truncated.body.copy()
test_keywords = test_data_truncated.keywords.copy()
test_keywords = pd.Series(list(map(lambda x: ast.literal_eval(x), test_keywords)))
test_keywords = list(map(lambda x: join_keywords(x), test_keywords))

In [ ]:
sample_title = data_small[:1]['title'].values[0]
sample_verdict = data_small[:1]['verdict'].values[0]
sample_body = data_small[:1]['body'].values[0]
# sample_keywords = data_small[:1]['keywords'].values[0]
sample_keywords = keywords[0]
print("Title:")
print(sample_title)
print("Verdict:")
print(sample_verdict)
print("Body:")
print(sample_body)
print("Keywords:")
print(sample_keywords)

Title:
aita for taking alcohol even if we all paid for it?
Verdict:
not the asshole
Body:
it was my friend's 25th birthday last week and she threw a party. lets call her mac. mac's always known to be the stingy one amongst our group of friends. she complains that she never has any money but she lives at home and spends money on expensive luxuries like botox, getting her hair and nails done etc. 

her actual birthday was on the thursday and the party was on the saturday. on thursday we took her to the movies and bought her flowers and her favourite ice-cream, tried to make her birthday really special for her. 

for her party she decided she wanted to have a cocktail-making party where we all made fancy cocktails and played board games. there were about 15 people going and she asked us all to pay her $30 the week prior to the party and she would buy the ingredients to make the cocktails. i suggested to her that we all just bring a bottle of something each but she said that would be unfai

In [ ]:
bs = 1
max_story = 800

In [ ]:
import torch
torch.manual_seed(67)
from torch.utils.data import Dataset 

class AitaDataset(Dataset):

  def __init__(self, titles, verdicts, txt_list, keywords, tokenizer, gpt2_type="gpt2", max_length=max_story):

    self.tokenizer = tokenizer # the gpt2 tokenizer we instantiated
    self.input_ids = []
    self.attn_masks = []
    self.labels = []

    for i in range(len(txt_list)):
      """
      This loop will iterate through each entry in the text corpus.
      For each bit of text it will prepend it with the start of text token,
      then append the end of text token and pad to the maximum length with the 
      pad token. 
      """

      encodings_dict = tokenizer('<|startofprompt|>'+ titles[i] + "<|endofprompt|>" 
                                 + verdicts[i] + "<|endofverdict|>"
                                 + keywords[i] + "<|eok|>"
                                 + txt_list[i] + "<|endoftext|>",
                                 truncation=True, 
                                 max_length=max_length, 
                                 padding="max_length",
                                 )
      prompts = tokenizer("<|startofprompt|>" + titles[i] + "<|endofprompt|>" 
                          + verdicts[i] + "<|endofverdict|>" + keywords[i] 
                          + "<|eok|>", truncation=True, max_length=max_length)
      
      """
      Each iteration then appends either the encoded tensor to a list,
      or the attention mask for that encoding to a list. The attention mask is
      a binary list of 1's or 0's which determine whether the langauge model
      should take that token into consideration or not. 
      """
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
      self.labels.append(torch.tensor(prompts["input_ids"]))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.labels[idx], self.attn_masks[idx] 

In [ ]:
from torch.utils.data import random_split

train_dataset = AitaDataset(train_titles, train_verdicts, train_keywords, train_stories, tokenizer, max_length=800)
test_dataset = AitaDataset(test_titles, test_verdicts, test_keywords, test_stories, tokenizer, max_length=800)

"""
# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

f'There are {train_size} samples for training, and {val_size} samples for validation testing'
"""

"\n# Split into training and validation sets\ntrain_size = int(0.9 * len(dataset))\nval_size = len(dataset) - train_size\n\ntrain_dataset, val_dataset = random_split(dataset, [train_size, val_size])\n\nf'There are {train_size} samples for training, and {val_size} samples for validation testing'\n"

In [ ]:
test_dataset = AitaDataset(titles, verdicts, stories, tokenizer, max_length=800)

In [ ]:
train_dataset[0][0].shape

torch.Size([800])

In [ ]:
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np

# Loading the model configuration and setting it to the GPT2 standard settings.
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# Create the instance of the model and set the token size embedding length
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# This step is optional but will enable reproducible runs.
seed_val = 67

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:
epochs = 3
warmup_steps = 1e2
sample_every = 500

''' Initializing Optimizer '''
from transformers import AdamW
# AdamW is a class from the huggingface library, it is the optimizer we will be using, and we will only be instantiating it with the default parameters. 
optimizer = AdamW(model.parameters(),
                  lr = 5e-4,
                  eps = 1e-8
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


validation_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), # Sampling for validation is sequential as the order doesn't matter.
            batch_size = bs
        )

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # Sampling for training is random
            batch_size = bs
        )

validation_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(test_dataset), # Sampling for validation is sequential as the order doesn't matter.
            batch_size = bs
        )

In [ ]:
from transformers import get_linear_schedule_with_warmup

"""
Total training steps is the number of data points, times the number of epochs. 
Essentially, epochs are training cycles, how many times each point will be seen by the model. 
"""

total_steps = len(train_dataloader) * epochs

"""
set a variable learning rate which will help scan larger areas of the 
problem space at higher LR earlier, then fine tune to find the exact model minima 
at lower LR later in training.
"""
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
import random
import time
import datetime

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    print(f'Beginning epoch {epoch_i + 1} of {epochs}')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_title = batch[1].to(device)
        b_masks = batch[2].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every 500 batches.
        if step % sample_every == 0 and not step == 0:

            PATH = "/content/drive/MyDrive/NLP_Proj/modelcheckpoints/keywords_model_1.pt"
            torch.save({
                        'step': step,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        }, PATH)    
                
            elapsed = format_time(time.time() - t0)
            print(f'Batch {step} of {len(train_dataloader)}. Loss:{batch_loss}. Time:{elapsed}')

            model.eval()
            # SAVING MODEL CHECKPOINT EVERY 500 STEPS
            sample_outputs = model.generate(
                                    input = b_title,
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print(f'Example output: {tokenizer.decode(sample_output, skip_special_tokens=True)}')
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print(f'Average Training Loss: {avg_train_loss}. Epoch time: {training_time}')

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[2].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids,  
                             attention_mask = b_masks,
                             labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print(f'Validation loss: {avg_val_loss}. Validation Time: {validation_time}')

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print(f'Total training took {format_time(time.time()-total_t0)}')

Beginning epoch 1 of 3


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 500 of 37512. Loss:2.3680431842803955. Time:0:01:28
Example output: wibta if i made a promise that i would never stop driving around the town? <|endofverdict|> not the asshole <|endofverdict|> my sister is driving and i live nearby, right around town. my parents always thought i wasn't going to ask the girls to come drive around the town with me because they always wanted to take photos of the cars to show them, but i always thought they were pretty bored to drive around with me. they seemed just really upset about having the drivers in their car. they weren't really on board with doing this, and just seemed really frustrated with me driving around every day, even though driving around all the time. she drove the car a total of 10 hours a day until she stopped paying a dime so she could drive it around town instead of just driving it around for the long night of a few days, because she never heard the sound of gas.

so, what do you think?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 1000 of 37512. Loss:1.5757266283035278. Time:0:03:00
Example output: wibta for calling my ex a bad person because he cheated on me and was so selfish? <|endofprompt|> asshole <|endofverdict|> first off, i'm not even sure what to make of my ex. he's a decent person, but really just a very selfish asshole, a bit of a bitch and a very passive aggressive asshole (especially since i feel like my ex basically never had the balls to get involved in any of my personal issues with me, but i feel like he made up a lot of lies and lies about me so i wouldn't really think to apologize to him anymore if it weren't for his actions. i understand he's a selfish and irresponsible guy, and i feel like he'd be like the best of anyone if i didn't come out against him even though i feel like i'm being petty, but i also feel like i'm being a shitty person, and that's just not appropriate anymore. 

but, my boyfriend, who we're all friends, and is


...

Batch 24500 of 37512. Loss:1.0736395120620728. Time:1:14:59
Example output: aita for being mad about what my ex told me? <|endofprompt|> not the asshole <|endofverdict|> my ex and i dated for 4 years. we had been together for 2 years before we broke up. we broke up on good terms after. i moved out of her house, and now live in a different city. i have been living with her for a while now.

we have talked more than one thing, and i don't see why she should tell me. i am happy and i love her but i don't want to take advantage of her happiness.

i have told her many times to get her own life and she always gets mad at me. she is very manipulative and i am very hurt by what i told her. i don't feel sorry for her at all. she is a very manipulative person. but she is really good friends with all the people that she talks to, and it's not like i am trying to say anything to her.

a couple nights


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 25000 of 37512. Loss:0.6591522693634033. Time:1:16:31
Example output: aita for thinking my friend is being childish? <|endofprompt|> not the asshole <|endofverdict|> my friend, t, is constantly messaging my ex (we're both juniors in high school).

she's always been emotionally absent, very nosy, and generally insecure about herself and other people, though she can get upset easily.

when she met me (and at one point she told me she liked me), she told me that she was depressed (she claims she's depressed), had a boyfriend, a little boy and a half and that she wanted to be with me, while also struggling with her depression and the guy. (she's been on and off since the divorce). 

i was not trying to sabotage either of them. she was more so upset that she just told me about my depression and how she found it hard to be with guys, or that she wanted me to lose myself for more than one night every two months. i told her she can try to lose weight


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Batch 25500 of 37512. Loss:0.7347431778907776. Time:1:18:02
Example output: aita for giving money to my daughter <|endofprompt|> not the asshole <|endofverdict|> i’m 50(f) and my two daughters (13f)  are finishing up high school and have been working in tech for 10 years now. i was in their school the other day and i decided to give them $400 in cash to help out with a project. i usually give her $10 in cash just so they can get their assignments. 
they are doing fine without me but my daughter is acting very selfishly towards me. she says she hates me and that she doesn’t want to see me. she even said she didn’t want to see me and that i gave it to her. 

so am i the asshole for giving this money to my daughter? i don’t like to give money in cash. it’s my money. <|eok|> giving money,daughter,aita


# **Metrics**

In [ ]:
torch.load('/content/drive/MyDrive/NLP_Proj/modelcheckpoints/keywords_model_1.pt').keys()

dict_keys(['step', 'model_state_dict', 'optimizer_state_dict'])

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/NLP_Proj/modelcheckpoints/keywords_model_1.pt')['model_state_dict'])

<All keys matched successfully>

In [ ]:
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/NLP_Proj/modelcheckpoints/keywords_model_1.pt')['optimizer_state_dict'])

In [ ]:
def calculate_perplexity(model, input_ids, attention_mask, pad_token, stride):
  max_len = model.config.n_positions
  loss_fn = torch.nn.CrossEntropyLoss()
  loss = []
  for i in range(0, input_ids.size(1), stride):
    begin = max(i+stride-max_len, 0)
    end = min(i+stride, input_ids.size(1))
    trg_len = end - i
    ip = input_ids[:, begin:end].to(device)
    mask = attention_mask[:, begin:end].to(device)
    target_ids = ip.clone()

    with torch.no_grad():
        output = model (ip,
                          labels=target_ids, 
                          attention_mask = mask,
                          token_type_ids=None
                        )
        l = output[0]*trg_len
    loss.append(l)
  
  ppl = torch.exp(torch.stack(loss).sum() / end)
  return ppl.item()

In [ ]:
perp = 0

for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_title = batch[1].to(device)
        b_masks = batch[2].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids,  
                             attention_mask = b_masks,
                             labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        #total_eval_loss += batch_loss
        perp += calculate_perplexity(model, b_input_ids, b_masks, tokenizer.convert_tokens_to_ids('<|pad|>'), 100)
        print(perp)


#avg_val_loss = total_eval_loss / len(validation_dataloader)

avg_perplexity = float(perp)/len(validation_dataloader)
    
validation_time = format_time(time.time() - t0)    

print(f'Perplexity: {avg_perplexity}')

6.839884281158447
19.86700677871704
27.6565580368042
38.257229804992676
43.7414345741272
57.95604372024536
66.9105896949768
83.24180269241333
93.76914167404175
98.52001190185547
105.45832109451294
125.56390428543091
140.20087480545044
146.98819780349731
165.5769715309143
173.96206903457642
182.98652410507202
191.46329164505005
194.9773805141449
207.0375955104828
217.41059947013855
221.4540297985077
238.35623049736023
250.7284438610077
256.8436734676361
263.70267367362976
276.32600474357605
281.199560880661
292.8910548686981
298.7321922779083
316.91979908943176
338.36301922798157
362.3727276325226
383.0516617298126
391.628901720047
421.8249065876007
429.619464635849
435.1171109676361
443.4346663951874
449.93537497520447
455.75472807884216
463.799772977829
478.17502665519714
500.27883791923523
505.9998314380646
513.1240766048431
522.4159381389618
526.0439567565918
536.7835369110107
543.3439245223999
548.6717162132263
553.8413505554199
572.8638401031494
577.0744180679321
588.393482208252


NameError: ignored

In [ ]:
avg_perplexity

11.454781072582314

# **BLEU**

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
bleu = 0

for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_title = batch[1].to(device)
        b_masks = batch[2].to(device)
        
        with torch.no_grad():        

  
            output = model.generate(input = b_title,
                           do_sample=True,   
                           top_k=50, 
                           max_length = 800,
                           top_p=0.95, 
                           num_return_sequences=1
                           )
          
            ref = tokenizer.decode(b_input_ids[0], skip_special_tokens=True).split()
            candidate = (tokenizer.decode(output[0], skip_special_tokens=True)).split()
            bleu += sentence_bleu(ref, candidate)
        


#avg_val_loss = total_eval_loss / len(validation_dataloader)

avg_bleu = float(bleu)/len(validation_dataloader)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-en

In [ ]:
len(validation_dataloader)

1002

In [ ]:
avg_bleu

0.36538703758430674